# [Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial)

.csv파일로 제출
id와 sentiment(0,1)

### [word2vec 참조](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/03/30/word2vec/)

### [nltk 참조](https://datascienceschool.net/view-notebook/118731eec74b4ad3bdd2f89bab077e1b/)
- 말뭉치 => 중지단어(뜻 없는 단어)사전 통해 중지단어 제거
- 토큰 생성
    - 긴 문자열을 작은 단위로 나눠짐(그 단위를 토큰이라 함)
- 형태소 분석
    - 형태소 : 의미가 있는 가장 작은 말의 단위
    - 어간 추출과 원형 복원
        - 어간 추출 : 같은 의미를 갖는 형태소의 기본형을 찾음
        - 원형 복원 : 같은 의미를 가지는 여러 단어를 사전형으로 통일
- 품사 태깅
    - 낱말을 문법적인 기능이나 형태, 뜻에 따라 구분

### part1 Bag Of Words
- HTML태그 제거 - BeautifulSoup
- 알파벳만 남김(숫자, 구두점 제거) - re
- 소문자로 변환, 공백으로 단어 구분
- 중지단어(의미없는 단어들)제거 - nltk

### part2 Word Vectors

### part3 More Fun With Word Vectors

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

<img src = "https://user-images.githubusercontent.com/28910538/54673538-cca58a80-4b3d-11e9-9bf1-e017c55a86f0.png" width = "500" height = "500" >

In [11]:
# header = 0 은 파일의 첫번 째 줄에 열 이름이 있음을 나타냄
# quating = 3은 파이썬에서 이중 따옴표를 무시
train_data = pd.read_csv("./data/movie/labeledTrainData.tsv", header=0, delimiter = "\t", quoting =3)
display(train_data.head(3))
print(train_data.shape)
print(train_data.columns.values)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


(25000, 3)
['id' 'sentiment' 'review']


In [24]:
example1 = BeautifulSoup(train_data["review"][0])
print(type(example1))
# .get_text를 통해 태그나 마크 없애줌
print(example1.get_text())

<class 'bs4.BeautifulSoup'>
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally

### [정규표현식 참조](https://wikidocs.net/4308)
![image](https://user-images.githubusercontent.com/28910538/54674482-dcbe6980-4b3f-11e9-9111-9846633bdb4a.png)


In [30]:
# 감정분석 위해 구두점,숫자 및 중지단어를 처리
# 알파벳이 아닌것들은 다 지움
letters_only = re.sub("[^a-zA-Z]", " ", example1.get_text())
print(type(letters_only))
print(letters_only)

<class 'str'>
 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is onl

In [33]:
# 다 소문자로
lower_case = letters_only.lower()
# 공백으로 단어 구분 
words = lower_case.split()
# 의미없는 단어들 (중지단어) 지우기 - nltk 통해
from nltk.corpus import stopwords
# 의미없는 중지단어들
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [35]:
# words는 위까지 전처리 과정 거치고 공백을 통해 단어로 구분된 list타입
# words in w => if not (w in stopword) => 이 w들은 wrods에 list로 저장
words = [w for w in words if not (w in stopwords.words("english"))]

In [37]:
# 위의 중지단어들이 다 날라감
print(words)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

In [45]:
def review_to_words(raw_review):
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review).get_text()
    # 2. 알파벳만 남김
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    # 3. 소문자로 다 바꾸고, 공백통해 단어로 구분
    words = letters_only.lower().split()
    # 4. 중지단어 제거
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not (w in stops)]
    # 5. list를 string 타입으로 변환(공백을 준)
    return (" ".join(meaningful_words))

In [49]:
clean_review = review_to_words(train_data["review"][0])
print(clean_review[:100])

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalke


In [52]:
clean_train_df = pd.DataFrame(columns = ["clean_review"])
clean_train_df["clean_review"] = train_data["review"]

In [72]:
%%time
clean_train_df["clean_review"] = clean_train_df["clean_review"].apply(review_to_words)

Wall time: 14.1 s


In [73]:
display(clean_train_df.head(3))

,clean_review
0,stuff going moment mj started listening music ...
1,classic war worlds timothy hines entertaining ...
2,film starts manager nicholas bell giving welco...


##### [apply 참조] (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

# Bag of Words
- 기계학습을 위한 숫자 표현으로 변환 방법

## Random Forest 사용
- pip install -U scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer